# EMNIST

In [ ]:
# Import dependencies
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import torchvision
import seaborn as sns
import numpy as np
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import torch
from tqdm import tqdm
from datetime import datetime
import yaml
import h5py
from copy import deepcopy
import ambiguous.models.cvae
from ambiguous.models.cvae import *
from ambiguous.dataset.dataset import partition_dataset

In [ ]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Seed for reproducibility
torch.manual_seed(42)
print(device)

In [ ]:
# Create the working directory from timestamp and model name
model_name = 'cvae_emnist' # invertible network
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
workingDir = f'tb_logs/{model_name}_{timestamp}'
os.mkdir(workingDir)

In [ ]:
lr = 1e-3
batch_size = 64
latent_dim = 4
TRAIN_CVAE = True

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])#,  transforms.Resize((32,32))]) # Dataset transform #  lambda x: (x>0.5).float(),
dataset = datasets.EMNIST(root='/share/datasets', download=False, split='byclass', train=True, transform=transform)
dataset = partition_dataset(dataset, range(10, 36))
train_set, val_set = torch.utils.data.random_split(dataset, [round(0.8*len(dataset)), round(0.2*len(dataset))])
test_set = datasets.EMNIST(root='/share/datasets', download=False, split='byclass', train=False, transform=transform)

# Dataloaders
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=12, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=12, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=12, shuffle=False)

In [ ]:
x,t=next(iter(train_loader))
torchvision.utils.save_image(next(iter(train_loader))[0], 'image.pdf', nrow=8)
print(['abcdefghijklmnopqrstuvwxyz'[i] for i in t])

In [ ]:
!tensorboard --logdir=./tb_logs --port=6006

In [ ]:
tb_logger = TensorBoardLogger('tb_logs', name=model_name)
enc_layers = [28*28, 1024, 1024]
dec_layers = [1024, 1024, 28*28]
if TRAIN_CVAE:
    model = EMNIST_CVAE(latent_dim, enc_layers, dec_layers, n_classes=26, conditional=True).to(device)
    trainer = pl.Trainer(gpus=1,logger=tb_logger,max_epochs=100,auto_lr_find=True)
    trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)